In [170]:
import pandas as pd
import numpy as np
df = pd.read_excel('/Users/sandeepk/Desktop/Verkada - Assessment.xlsx')
df.head(3)

,Client Status,Source,Created Date,Unnamed: 3,Unnamed: 4
0,SQL,Email,2019-04-01,1,NaN
1,SQL,Email,2019-04-01,1,NaN
2,SQL,Email,2019-04-01,1,NaN


##  Verkada Answer
- libraries used: `pandas` `numpy`

In [153]:
import pandas as pd
import numpy as np

In [5]:
#Function outputs weekly conversion rate
def SQL_MQL_Conv(csv_line:str):
    df = pd.read_excel(csv_line)
    df['Day'] = df['Created Date'].dt.strftime('%d')
    conditions = [(df["Client Status"] == 'SQL'),
                 ]
    values = [1]

    conditions1 = [(df["Client Status"] == 'MQL'),
                 ]
    values1  = [1]

    df['SQL'], df['MQL'] = np.select(conditions, values), np.select(conditions1, values1)

    aggs = df.groupby('Day').agg({'SQL': 'sum', 'MQL': 'sum'})

    weekly_ls = []
    val, val2 = 0, 7
    while val2 <= len(aggs)+1:
        weekly_ls.append(sum(aggs['SQL'].iloc[val:val2])/ sum(aggs['MQL'].iloc[val:val2])*100)
        val += 7
        val2 += 7
    verkada_df = pd.DataFrame()
    verkada_df['Conversion Rate'] = [f'%{round(val,2)}' for val in weekly_ls]
    verkada_df.rename(index={0:"Week 1", 1: 'Week 2', 2: 'Week 3', 3: 'Week 4'}, inplace=True)
    return verkada_df

SQL_MQL_Conv('/Users/Sandeepk/Desktop/Verkada - Assessment.xlsx')


,Conversion Rate
Week 1,%88.24
Week 2,%88.89
Week 3,%50.55
Week 4,%64.39


### Conversion Viz

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel('/Users/sandeepk/Desktop/Verkada - Assessment.xlsx')
df['Day'] = df['Created Date'].dt.strftime('%d')

conditions = [(df["Client Status"] == 'SQL'),
             (df['Client Status'] == 'MQL'),
             ]

values = [1,0]



aggs = df.groupby('Day').agg({'Conversion': ['count', 'sum']})#['Client Status'].iloc[0:3]
aggs['Rate'] = (aggs[('Conversion', 'sum')]/aggs[('Conversion', 'count')]).round(3)
aggs

Conversion        Rate
         count  sum       
Day                       
01          19    9  0.474
02           7    4  0.571
03          48   24  0.500
04           7    2  0.286
05           5    1  0.200
06           8    4  0.500
07           2    1  0.500
08           4    0  0.000
09           7    5  0.714
10          56   41  0.732
11          17    5  0.294
12          27    1  0.037
13          24   11  0.458
14           1    1  1.000
15          92   36  0.391
16          64   23  0.359
17          86   22  0.256
18          85   23  0.271
19          53   20  0.377
20          24   10  0.417
21           4    3  0.750
22          98   26  0.265
23           5    2  0.400
24         122   41  0.336
25         381  138  0.362
26         473  217  0.459
27         440  171  0.389

In [251]:
import plotly.express as px
fig = px.line(aggs, x=aggs.index, y='Rate', title='SQL Conversion Rate - April 2022')
fig.update_layout(template="plotly_dark")
fig.show()
#fig.write_image('/Users/sandeepk/Desktop/Verkata_SQLConversion.png')

fig.write_html("/Users/sandeepk/Desktop/Verkata_SQLConversion.html")

In [27]:
fig = px.line(x=['Week 1', 'Week 2', 'Week 3', 'Week 4'],\
              y=[float(conv.split('%')[1])/ 100 for conv in z['Conversion Rate']],\
              title='SQL Conversion Rate - April 2022')
fig.show()

In [16]:
#Preprocessing
z = SQL_MQL_Conv('/Users/Sandeepk/Desktop/Verkada - Assessment.xlsx')
[float(conv.split('%')[1])/ 100 for conv in z['Conversion Rate']]

[0.8824, 0.8889, 0.5055, 0.6439]

In [20]:
z = SQL_MQL_Conv('/Users/Sandeepk/Desktop/Verkada - Assessment.xlsx')

week_df = pd.DataFrame()
week_df['Weekly Rate'] = [float(conv.split('%')[1])/ 100 for conv in z['Conversion Rate']]
week_df

,Weekly Rate
0,0.8824
1,0.8889
2,0.5055
3,0.6439


In [28]:
df = pd.read_excel('/Users/sandeepk/Desktop/Verkada - Assessment.xlsx')
df['Day'] = df['Created Date'].dt.strftime('%d')
conditions = [(df["Client Status"] == 'SQL'),
             ]
values = [1]

conditions1 = [(df["Client Status"] == 'MQL'),
             ]
values1  = [1]

df['SQL'], df['MQL'] = np.select(conditions, values), np.select(conditions1, values1)

In [34]:
#Daily count of SQL & MQL
sql_mql_aggs = df.groupby('Day').agg({'SQL': sum, 'MQL': sum})

sql_ls, mql_ls = [],[]
start, end = 0, 7
while end <= len(sql_mql_aggs)+1:
    sql_ls.append(sum(sql_mql_aggs['SQL'].iloc[start:end]))
    mql_ls.append(sum(sql_mql_aggs['MQL'].iloc[start:end]))
    start += 7
    end += 7
    

In [362]:
fig = px.line(sql_mql_aggs,
              x="Day",
              y=["MQL", "SQL"],
              labels={
                     "variable": "Contract Type",
                     "Day": "Day",
                     "value": "Count"
                 },
              title='SQL Vs MQL Count by Day')
fig.update_layout(template="plotly_dark")

fig.show()

fig.write_html("/Users/sandeepk/Desktop/SQL_VS_MQL_Daily.html")

### viz weekly

In [84]:
sql_mql_aggs['Day'] = sql_mql_aggs.index

#Weekly count of SQL VS MQL
bar_df = pd.DataFrame()
bar_df['SQL'], bar_df['MQL'] = sql_ls, mql_ls
bar_df.rename(index = {0:'Week 1', 1: 'Week 2', 2: 'Week 3', 3: 'Week 4'}, inplace=True)
bar_df['Week'] = bar_df.index
bar_df

,SQL,MQL,Week
Week 1,45,51,Week 1
Week 2,64,72,Week 2
Week 3,137,271,Week 3
Week 4,595,924,Week 4


In [207]:
sql = bar_df[['SQL', 'Week']]
mql = bar_df[['MQL', 'Week']]
#Run function below before
sql['Percent Change'] = sql_change
mql['Percent Change'] = mql_change

sql, mql = sql.rename(columns = {'SQL': 'Count'}), mql.rename(columns = {'MQL': 'Count'}),#, inplace = True)
barplot_df = sql.append(mql)
barplot_df['Contract Type'] = \
['SQL' for val in range(int(len(barplot_df)/2))] + ['MQL' for val in range(int(len(barplot_df)/2))]

barplot_df

/var/folders/2n/y8fl7fr16hs_g_085s9fnpqm0000gn/T/ipykernel_8404/1202520345.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Count,Week,Percent Change,Contract Type
Week 1,45,Week 1,0,SQL
Week 2,64,Week 2,+ %42.22,SQL
Week 3,137,Week 3,+ %114.06,SQL
Week 4,595,Week 4,+ %334.31,SQL
Week 1,51,Week 1,0,MQL
Week 2,72,Week 2,+ %41.18,MQL
Week 3,271,Week 3,+ %276.39,MQL
Week 4,924,Week 4,+ %240.96,MQL


In [201]:
def percent_change(df):
    start,end = 0,1
    change = ['0']
    while end < len(df):
        if (df['Count'].iloc[end]- df['Count']).iloc[start]/df['Count'].iloc[start]*100 > 0:
            change.append(\
                          f'+ %{round((df["Count"].iloc[end]- df["Count"]).iloc[start]/df["Count"].iloc[start]*100, 2)}')

        else:
            change.append(\
                          f'- %{round((df["Count"].iloc[end]- df["Count"]).iloc[start]/df["Count"].iloc[start]*100, 2)}')

        end+=1
        start+=1
    return change

sql_change = percent_change(sql)
mql_change = percent_change(mql)

In [202]:
sql_change

['0', '+ %42.22', '+ %114.06', '+ %334.31']

In [208]:
fig = px.bar(barplot_df, x='Week',
             y = 'Count',
             color = 'Contract Type',
             text = 'Percent Change',
             title = 'SQL Vs MQL Count by Week'
            )

fig.update_layout(template="plotly_dark")
fig.show()
fig.write_html("/Users/sandeepk/Desktop/SQL_VS_MQL_Weekly.html")

## By Category

In [210]:
df = pd.read_excel('/Users/sandeepk/Desktop/Verkada - Assessment.xlsx')
df['Day'] = df['Created Date'].dt.strftime('%d')
conditions = [(df["Client Status"] == 'SQL'),
             ]
values = [1]

conditions1 = [(df["Client Status"] == 'MQL'),
             ]
values1  = [1]

df['SQL'], df['MQL'] = np.select(conditions, values), np.select(conditions1, values1)

In [228]:
dd[4:]

SQL  MQL
Client Status Source               
SQL           Direct Mail  126    0
              Email        243    0
              Other          9    0
              Paid Social  463    0

In [225]:
dd = df.groupby(['Client Status','Source']).agg({'SQL': sum, 'MQL': sum})
dd['MQL'].iloc[0:4]

Client Status  Source     
MQL            Direct Mail     208
               Email            50
               Other            11
               Paid Social    1049
Name: MQL, dtype: int64

In [237]:
[dd[1] for dd in dd.index[0:4]]

['Direct Mail', 'Email', 'Other', 'Paid Social']

In [242]:
dd['SQL'].iloc[0:4]

Client Status  Source     
MQL            Direct Mail    0
               Email          0
               Other          0
               Paid Social    0
Name: SQL, dtype: int64

In [267]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='MQL', x=[dd[1] for dd in dd.index[0:4]], y=dd['MQL'].iloc[0:4]),
    go.Bar(name='SQL', x=[dd[1] for dd in dd.index[4:]], y=dd['SQL'].iloc[4:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='SQL Vs MQL by Category',
                 xaxis_title="Source",
                 yaxis_title="Count")

fig.update_layout(template="plotly_dark")

fig.show()
fig.write_html("/Users/sandeepk/Desktop/SQL_VS_MQL_byCategory.html")

In [273]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Scatter(name='MQL', x=[dd[1] for dd in dd.index[0:4]], y=dd['MQL'].iloc[0:4],
              mode = 'markers', marker_size=[70, 60, 21, 100]),
    go.Scatter(name='SQL', x=[dd[1] for dd in dd.index[4:]], y=dd['SQL'].iloc[4:],
              mode = 'markers', marker_size=[58, 74, 19, 88])
    
])
fig.update_layout(title_text='SQL Vs MQL by Category',
                 xaxis_title="Source",
                 yaxis_title="Count")
fig.update_layout(template="plotly_dark")
fig.show()
#fig.write_html("/Users/sandeepk/Desktop/SQL_VS_MQL_byCategory_bubble.html")

### Lambda + numpy

In [299]:
import random
import numpy as np

In [306]:
ids = [x for x in range(10)]
nums = random.sample(range(10, 30), 10)

ls =[]



np.argmax(nums), nums

(5, [20, 12, 14, 26, 15, 28, 16, 13, 23, 24])

In [335]:
top_2_idx = list(np.argsort(nums)[-2:])#.apply(lamda i: [-2:]
#[ids[i] for i in top_2_idx]
top_2_idx

[3, 5]

In [372]:
ids = list(range(0,5))
a = [5,3,1,4,10]
num = 2
sorted(range(len(a)), key=lambda i: a[i], reverse=True)[:2]#, reverse=True)

[4, 0]

In [361]:
import random

In [354]:
z = random.sample(range(10, 30), 5)
a, z

([5, 3, 1, 4, 10], [27, 25, 16, 28, 26])

In [359]:
sorted(a, key = lambda i: i*7<z[0]) #, reverse = True

[5, 4, 10, 3, 1]